# Boosting 이란
- 여러 개의 약한 Decision Tree를 조합해서 사용하는 Ensemble 기법중 하나이다.
- 약한 예측 모델들의 학습 에러에 가중치를 두고, 순차적으로 다음 학습 모델에 반영하여 강한 예측모델 제작이 목표

# GBM(Gradient Boosting Machine)
- GBM은 여러개의 weak learner를 순차적으로 학습, 예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 계산해서 학습
- 가중치의 업데이트 방법은 <strong>경사 하강법</strong>을 사용

# XGBoost란
- XGBoost는 Extreme Gradient Boosting의 약자
- Boosting 기법을 이용하여 구현한 알고리즘은 Gradient Boost가 대표적인데 이 알고리즘을 병렬학습이 지원되도록 구현한 라이브러리
- 분류, 회귀 문제를 모두 지원하며, 성능과 효율이 좋아서 인기가 높다
- XGBoost 장점
    - GBM 대비 빠른 수행시간 : 병렬 처리로 학습, 분류속도가 빠름
    - 과적합 규제 : 표준 GBM의 경우 과적합 규제기능이 없으나, XGBoost는 과적합 규제 기능이 있어서 강한 내구성 보유
    - 분류와 회귀 영역에서 뛰어난 예측성능
        - CART(Classifier and Regression Tree)앙상블 모델을 사용한다.
    - 조기종료 기능
    - 다양한 옵션 제공 및 커스텀이 용이

## 필수 라이브러리

In [52]:
import numpy as np  # 넘파이
import pandas as pd  # 판다스
import seaborn as sns  # 시본
import matplotlib as mpl
import matplotlib.pyplot as plt  # matplotlib
import warnings
import xgboost as xgb
from scipy.special import expit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
warnings.simplefilter(action='ignore', category=FutureWarning)
%config Completer.use_jedi = False
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

mpl.rc('font', family='D2coding')
mpl.rc('axes', unicode_minus=False)
sns.set(font='D2coding', rc={'axes.unicode_minus': False}, style='darkgrid')
plt.rc('figure', figsize=(10, 8))

# 로지스틱 회귀와 평가지표

## 데이터셋 - 위스콘신 유방암 예측

In [2]:
!pip install xgboost

     ---------------------------------------- 89.1/89.1 MB 4.0 MB/s eta 0:00:00


In [4]:
cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [9]:
X = cancer.data
y = cancer.target

cancer_df = pd.DataFrame(X, columns=cancer.feature_names)
cancer_df['target'] = y
cancer_df.head(1)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189,0


In [12]:
print('data shape :', X.shape)
print('target shape :', y.shape)

data shape : (569, 30)
target shape : (569,)


In [15]:
# 양성 데이터 세트 갯수와 음성 데이터 세트 개수의 파악
np.unique(cancer.target, return_counts=True)

(array([0, 1]), array([212, 357], dtype=int64))

In [16]:
# feature 확인
for i, feature in enumerate(cancer.feature_names):
    print(f'feature{(i+1)} : ', feature)

feature1 :  mean radius
feature2 :  mean texture
feature3 :  mean perimeter
feature4 :  mean area
feature5 :  mean smoothness
feature6 :  mean compactness
feature7 :  mean concavity
feature8 :  mean concave points
feature9 :  mean symmetry
feature10 :  mean fractal dimension
feature11 :  radius error
feature12 :  texture error
feature13 :  perimeter error
feature14 :  area error
feature15 :  smoothness error
feature16 :  compactness error
feature17 :  concavity error
feature18 :  concave points error
feature19 :  symmetry error
feature20 :  fractal dimension error
feature21 :  worst radius
feature22 :  worst texture
feature23 :  worst perimeter
feature24 :  worst area
feature25 :  worst smoothness
feature26 :  worst compactness
feature27 :  worst concavity
feature28 :  worst concave points
feature29 :  worst symmetry
feature30 :  worst fractal dimension


## 표준화

In [17]:
scaler = StandardScaler()
scaled = scaler.fit_transform(cancer_df.drop('target', axis=1))

## 데이터 분리

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    scaled, cancer_df.target, test_size=0.2, random_state=156)

### 로지스틱 회귀

In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [21]:
# 훈련 모델을 이용하여 5개의 샘플 예측
print(lr.predict(X_train[:5]))

[0 1 1 1 1]


In [22]:
print(lr.predict_proba(X_train[:5]))

[[9.90703488e-01 9.29651217e-03]
 [3.95649332e-02 9.60435067e-01]
 [6.71031350e-06 9.99993290e-01]
 [1.61627147e-04 9.99838373e-01]
 [8.73555079e-02 9.12644492e-01]]


In [23]:
# classes_ 속성
print(lr.classes_)

[0 1]


In [24]:
print(lr.coef_, lr.intercept_)

[[-0.4721854  -0.38258173 -0.44454406 -0.54316124 -0.14319742  0.72706363
  -0.70746211 -0.96418695 -0.07905176  0.27001329 -1.19658305  0.19765208
  -0.5890956  -0.95010481 -0.31150744  0.75959144  0.14533365 -0.32546344
   0.33243796  0.4394735  -0.86822466 -1.27824666 -0.65146591 -0.90866007
  -0.44631307  0.16705142 -0.8564404  -0.83122684 -1.0734741  -0.54801513]] [0.21706473]


In [26]:
# 예측한 5개의 z값 출력
decisions = lr.decision_function(X_train[:5])
decisions

array([-4.66877599,  3.18944318, 11.91185818,  8.7300568 ,  2.34636033])

In [27]:
print(expit(decisions))

[0.00929651 0.96043507 0.99999329 0.99983837 0.91264449]


In [28]:
# 예측하기
predict = lr.predict(X_test)

### 평가지표
1. 회귀 : MAE, MSE, RMSE, SSE, ...
2. 분류 : 정확도, 정밀도, 재현율(민감도), F1 score, ROC 커브
3. 클러스터링 : DBI, 엘보우 메서드, 실루엣계수

In [29]:
# 정확도(Accuracy)
# 이진 분류일 경우 데이터에 모델의 성능이 왜곡될 수 있다.
# 예측결과가 동일한 데이터 수 / 전체 예측 데이터 수
accuracy_score(y_test, predict)

0.9736842105263158

### Confusion Matrix

In [32]:
cm = confusion_matrix(y_test, predict)
cm

array([[35,  2],
       [ 1, 76]], dtype=int64)

In [42]:
TN, FP, FN, TP = cm.ravel()

#### 정밀도(Precision)
- True Positive / All Positive

In [43]:
precision = TP / (TP + FP)
precision

0.9743589743589743

In [45]:
precision_score(y_test, predict)

0.9743589743589743

#### 재현율(Recall), 민감도
- 실제 Positiove인 전체 건수에서 옳게 예측한 건수의 비밀
- TPR(True Positive

In [46]:
recall = TP / (FN+TP)
recall

0.987012987012987

In [47]:
from sklearn.metrics import recall_score
recall_score(y_test, predict)

0.987012987012987

#### F1 Score
- 재현율과 정밀도의 균을 나타내는 수치

In [48]:
f1 = 2 * (precision * recall) / (precision + recall)
f1

0.9806451612903225

In [49]:
from sklearn.metrics import f1_score
f1_score(y_test, predict)

0.9806451612903225